In [1]:
#imports et configuration
import tensorflow as tf
import numpy as np
import cv2
import os
import pathlib

# Vérification que tout va bien
print("TensorFlow version:", tf.__version__)
print("GPU disponible ?", tf.config.list_physical_devices('GPU'))

# Chemins des dossiers
DATA_DIR = "../data/calibration"  # Vos 170 images
MODELS_DIR = "../models"          # Où on va sauvegarder
os.makedirs(MODELS_DIR, exist_ok=True)

2025-12-08 17:54:41.141455: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-08 17:54:41.381168: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-08 17:54:41.581215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765212881.745917   13879 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765212881.793713   13879 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-08 17:54:42.220353: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

TensorFlow version: 2.18.0
GPU disponible ? []


W0000 00:00:1765212890.715832   13879 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
#Téléchargement du Modèle Géant (SSD MobileNet V2)
import glob

# 1. On définit un dossier de cache ABSOLU (pas de "..")
# On le met dans le dossier 'data' de votre projet
cache_dir = os.path.abspath("../data/tf_cache")
os.makedirs(cache_dir, exist_ok=True)

print(f"Téléchargement vers : {cache_dir}")

model_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

# 2. Téléchargement
# Keras va extraire les fichiers ici
path_to_tar = tf.keras.utils.get_file(
    fname="ssd_mobilenet_v2.tar.gz",
    origin=model_url,
    untar=True,
    cache_dir=cache_dir,
    cache_subdir="." # On extrait directement dans tf_cache
)

# 3. RECHERCHE AUTOMATIQUE
# On ne devine pas le nom, on demande à Python de trouver le dossier "saved_model"
# qui se trouve quelque part dans cache_dir
found_paths = glob.glob(os.path.join(cache_dir, "**", "saved_model"), recursive=True)

if found_paths:
    # On prend le premier qu'on trouve
    saved_model_path = found_paths[0]
    print(f"\n✅ SUCCÈS ! Modèle trouvé ici :")
    print(saved_model_path)
    
    # Vérification du contenu (on doit voir saved_model.pb)
    print("Contenu :", os.listdir(saved_model_path))
else:
    print("\n❌ ERREUR CRITIQUE : Impossible de trouver le dossier 'saved_model' après extraction.")

Téléchargement vers : /home/viet/projet-10-edge-vision/data/tf_cache
20515344/20515344 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

✅ SUCCÈS ! Modèle trouvé ici :
/home/viet/projet-10-edge-vision/data/tf_cache/ssd_mobilenet_v2_extracted/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model
Contenu : ['saved_model.pb', 'variables']


In [6]:
#Conversion "Classique" (Float32)
# 1. Convertir en TFLite standard (Float32)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()

# Sauvegarder
float_model_path = os.path.join(MODELS_DIR, "model_float32.tflite")
with open(float_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Modèle Float32 sauvegardé : {float_model_path}")
print(f"Taille : {len(tflite_model) / 1024 / 1024:.2f} MB")

W0000 00:00:1765213406.315246   13879 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1765213406.315330   13879 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-12-08 18:03:26.316662: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /home/viet/projet-10-edge-vision/data/tf_cache/ssd_mobilenet_v2_extracted/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model
2025-12-08 18:03:26.365018: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-12-08 18:03:26.365051: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /home/viet/projet-10-edge-vision/data/tf_cache/ssd_mobilenet_v2_extracted/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model
I0000 00:00:1765213406.739812   13879 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-12-08 18:03:26.772439: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bund

Modèle Float32 sauvegardé : ../models/model_float32.tflite
Taille : 11.68 MB


In [9]:
#Préparation images pour la Calibration (Crucial !)
def representative_dataset():
    files = os.listdir(DATA_DIR)
    
    # On prend 100 images
    for file_name in files[:100]:
        img_path = os.path.join(DATA_DIR, file_name)
        img = cv2.imread(img_path)
        
        if img is not None:
            img = cv2.resize(img, (320, 320))
            
            # --- CORRECTION ICI ---
            # On garde le format UINT8 (Entiers de 0 à 255)
            # Au lieu de convertir en Float32
            input_data = np.expand_dims(img, axis=0).astype(np.uint8)
            
            yield [input_data]
            
print("✅ Générateur de dataset prêt (Mode UINT8) !")

✅ Générateur de dataset prêt (Mode UINT8) !


In [10]:
#La Conversion "Optimisée" (Int8 Quantization)
# Recharger le convertisseur
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)

# --- ACTIVATION DES OPTIMISATIONS ---
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# On lui donne vos images pour qu'il apprenne l'échelle des valeurs
converter.representative_dataset = representative_dataset

# On force les opérations en entiers (Int8) là où c'est possible
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# On force aussi les entrées/sorties en entiers (Full Integer Quantization)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

try:
    tflite_quant_model = converter.convert()
    
    # Sauvegarder
    quant_model_path = os.path.join(MODELS_DIR, "model_int8.tflite")
    with open(quant_model_path, 'wb') as f:
        f.write(tflite_quant_model)

    print(f"Modèle Int8 sauvegardé : {quant_model_path}")
    print(f"Taille : {len(tflite_quant_model) / 1024 / 1024:.2f} MB")

except Exception as e:
    print("Erreur pendant la quantification :", e)

W0000 00:00:1765213784.069534   13879 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1765213784.069610   13879 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-12-08 18:09:44.069794: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /home/viet/projet-10-edge-vision/data/tf_cache/ssd_mobilenet_v2_extracted/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model
2025-12-08 18:09:44.110955: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-12-08 18:09:44.110987: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /home/viet/projet-10-edge-vision/data/tf_cache/ssd_mobilenet_v2_extracted/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model
2025-12-08 18:09:44.436794: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-12-08 18:09:45.180051: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel b

Modèle Int8 sauvegardé : ../models/model_int8.tflite
Taille : 4.07 MB


In [11]:
#Officialiser dans MLflow
import mlflow

# On définit le nom de l'expérience
mlflow.set_experiment("Projet_10_Edge_Optimization")

with mlflow.start_run(run_name="MobileNetV2_Quantization"):
    # 1. On enregistre les métriques (La preuve que ça marche)
    mlflow.log_metric("size_float32_mb", 11.68)
    mlflow.log_metric("size_int8_mb", 4.07)
    mlflow.log_metric("compression_ratio", 11.68 / 4.07)
    
    # 2. On sauvegarde les fichiers modèles DANS MLflow (Le Registry)
    mlflow.log_artifact(float_model_path, artifact_path="models")
    mlflow.log_artifact(quant_model_path, artifact_path="models")
    
    print("✅ Modèles et métriques envoyés à MLflow !")

/home/viet/projet-10-edge-vision/venv/lib/python3.12/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251
2025/12/08 18:14:59 INFO mlflow.tracking.fluent: Experiment with name 'Projet_10_Edge_Optimization' does not exist. Creating a new experiment.


✅ Modèles et métriques envoyés à MLflow !
